In [3]:
from sklearn.datasets import load_files
categories = ['alt.atheism', 'soc.religion.christian',
              'comp.graphics', 'sci.med']
# twenty_train = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42)
twenty_train = load_files(container_path='C:/Users/Tran/git/_learning/learning_ml/data/20news-bydate/20news-bydate-train', categories=categories)

In [4]:
twenty_train.target_names

['alt.atheism', 'comp.graphics', 'sci.med', 'soc.religion.christian']

In [5]:
len(twenty_train.data)


2257

In [6]:
twenty_train.data[0]


'From: dpc47852@uxa.cso.uiuc.edu (Daniel Paul Checkman)\nSubject: Re: Is MSG sensitivity superstition?\nArticle-I.D.: news.C5wI4F.Dt\nOrganization: University of Illinois at Urbana\nLines: 22\n\nbruce@Data-IO.COM (Bruce Reynolds) writes:\n\n>Anecedotal evidence is worthless.  Even doctors who have been using a drug\n>or treatment for years, and who swear it is effective, are often suprised\n>at the results of clinical trials.  Whether or not MSG causes describable,\n>reportable, documentable symptoms should be pretty simple to discover.  \n\nI tend to disagree- I think anecdotal evidence, provided there is a lot of it,\nand it is fairly consistent, will is very important.  First, it points to the\nnecessity of doing a study, and second, it at least says that the effects are\nall psychological (or possibly allergy in this case).  As I\'ve pointed out \nbefore, pyschological effects are no less real than other effects.  One       \nperson\'s "make-believe" can easily be another person\'s

In [7]:
len(twenty_train.target)


2257

In [8]:
print("\n".join(["test1", "test2"]))
print("\n".join(twenty_train.data[0].split("\n")[:3]))


test1
test2
From: dpc47852@uxa.cso.uiuc.edu (Daniel Paul Checkman)
Subject: Re: Is MSG sensitivity superstition?
Article-I.D.: news.C5wI4F.Dt


In [9]:
twenty_train.target[:10]

array([2, 1, 3, 2, 0, 2, 2, 1, 1, 0])

In [10]:
for t in twenty_train.target[:10]:
    print(twenty_train.target_names[t])

sci.med
comp.graphics
soc.religion.christian
sci.med
alt.atheism
sci.med
sci.med
comp.graphics
comp.graphics
alt.atheism


In [11]:
twenty_train.filenames[:10]

array([ 'C:/Users/Tran/git/_learning/learning_ml/data/20news-bydate/20news-bydate-train\\sci.med\\59184',
       'C:/Users/Tran/git/_learning/learning_ml/data/20news-bydate/20news-bydate-train\\comp.graphics\\38735',
       'C:/Users/Tran/git/_learning/learning_ml/data/20news-bydate/20news-bydate-train\\soc.religion.christian\\20968',
       'C:/Users/Tran/git/_learning/learning_ml/data/20news-bydate/20news-bydate-train\\sci.med\\58152',
       'C:/Users/Tran/git/_learning/learning_ml/data/20news-bydate/20news-bydate-train\\alt.atheism\\51249',
       'C:/Users/Tran/git/_learning/learning_ml/data/20news-bydate/20news-bydate-train\\sci.med\\58047',
       'C:/Users/Tran/git/_learning/learning_ml/data/20news-bydate/20news-bydate-train\\sci.med\\58986',
       'C:/Users/Tran/git/_learning/learning_ml/data/20news-bydate/20news-bydate-train\\comp.graphics\\38727',
       'C:/Users/Tran/git/_learning/learning_ml/data/20news-bydate/20news-bydate-train\\comp.graphics\\38596',
       'C:/Users/

In [13]:
from sklearn.feature_extraction.text import CountVectorizer
count_vect = CountVectorizer(decode_error='ignore')
X_train_counts = count_vect.fit_transform(twenty_train.data)
X_train_counts.shape

(2257, 35787)

In [14]:
from sklearn.feature_extraction.text import TfidfTransformer
# tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
tf_transformer = TfidfTransformer(use_idf=False).fit(X_train_counts)
X_train_tf = tf_transformer.transform((X_train_counts))
X_train_tf.shape


(2257, 35787)

In [15]:
tfidf_transformer = TfidfTransformer()
X_train_tfidf = tfidf_transformer.fit_transform(X_train_counts)
X_train_tfidf.shape


(2257, 35787)

In [19]:
from modeldb.sklearn_native.ModelDbSyncer import *
from modeldb.sklearn_native import SyncableMetrics

# modeldb start

name = "simple sample 2"
author = "srinidhi"
description = "simple LR for credit default prediction"
syncer_obj = Syncer(
    NewOrExistingProject(name, author, description),
    DefaultExperiment(),
    NewExperimentRun("credit test"))

# modeldb end

from sklearn.naive_bayes import MultinomialNB
clf = MultinomialNB()
clf.fit_sync(X_train_tf, twenty_train.target)

print(clf)

AttributeError: 'MultinomialNB' object has no attribute 'fit_sync'

In [20]:
docs_new = ['God is love', 'OpenGL on the GPU is fast']
X_new_counts = count_vect.transform(docs_new)
X_new_tfidf = tfidf_transformer.transform(X_new_counts)
predicted = clf.predict_sync(X_new_tfidf)
for doc, cat in zip(docs_new, predicted):
    print('%r > %s' % (doc, twenty_train.target_names[cat]))

# modeldb start
score = SyncableMetrics.compute_metrics(
    clf, accuracy_score, X_new_counts, predicted, X_train_tfidf, "features",
    'default payment next month')
# modeldb end

# modeldb start
syncer_obj.sync()
# modeldb end


PandasError: DataFrame constructor not properly called!

In [22]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer(decode_error='ignore')), ('tfidf', TfidfTransformer()), ('clf', MultinomialNB())])
text_clf = text_clf.fit(twenty_train.data, twenty_train.target)


In [24]:
import numpy as np
twenty_test = load_files(container_path='C:/Users/Tran/git/_learning/learning_ml/data/20news-bydate/20news-bydate-test', categories=categories)
docs_test = twenty_test.data
predicted = text_clf.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.83488681757656458

In [72]:
from sklearn.linear_model import SGDClassifier
text_clf_2 = Pipeline([('vect', CountVectorizer(decode_error='ignore')), ('tfidf', TfidfTransformer()), 
                       ('clf', SGDClassifier(loss='hinge', penalty='l2', alpha=1e-3, n_iter=5, random_state=42))])
text_clf_2 = text_clf_2.fit(twenty_train.data, twenty_train.target)
predicted = text_clf_2.predict(docs_test)
np.mean(predicted == twenty_test.target)

0.91344873501997337

In [73]:
from sklearn import  metrics
print(metrics.classification_report(twenty_test.target, predicted, target_names=twenty_test.target_names))

                        precision    recall  f1-score   support

           alt.atheism       0.95      0.82      0.88       319
         comp.graphics       0.87      0.98      0.92       389
               sci.med       0.95      0.89      0.92       396
soc.religion.christian       0.90      0.94      0.92       398

           avg / total       0.92      0.91      0.91      1502



In [74]:
metrics.confusion_matrix(twenty_test.target, predicted)

array([[261,  10,  14,  34],
       [  4, 381,   2,   2],
       [  4,  34, 354,   4],
       [  6,  12,   4, 376]])

In [78]:
from sklearn.model_selection import GridSearchCV
parameters = {'vect__ngram_range': [(1,1), (1,2)], 'tfidf__use_idf': (True, False), 'clf__alpha': (1e-2, 1e-3)}
gs_clf = GridSearchCV(text_clf_2, parameters, n_jobs=-1)


In [79]:
gs_clf = gs_clf.fit(twenty_train.data[:400], twenty_train.target[:400])


In [80]:
twenty_train.target_names[gs_clf.predict(['God is love'])[0]]

'soc.religion.christian'

In [81]:
gs_clf.best_score_

0.92749999999999999

In [82]:
gs_clf.best_estimator_

Pipeline(steps=[('vect', CountVectorizer(analyzer=u'word', binary=False, decode_error='ignore',
        dtype=<type 'numpy.int64'>, encoding=u'utf-8', input=u'content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        str...     penalty='l2', power_t=0.5, random_state=42, shuffle=True, verbose=0,
       warm_start=False))])

In [83]:
gs_clf.best_params_

{'clf__alpha': 0.001, 'tfidf__use_idf': True, 'vect__ngram_range': (1, 1)}